In [29]:
import pandas as pd
import numpy as np
import sys
import json
import jsonlines
from sklearn.model_selection import train_test_split

### Data for language model (LM)

In [30]:
!mkdir ../../age
!mkdir ../../age/lm
!mkdir ../../age/target_clf
!mkdir ../../age/substitute_clf/

mkdir: cannot create directory ‘../../ageN’: File exists
mkdir: cannot create directory ‘../../ageN/lm’: File exists
mkdir: cannot create directory ‘../../ageN/target_clf’: File exists
mkdir: cannot create directory ‘../../ageN/substitute_clf/’: File exists


In [31]:
data = pd.read_csv('../data/age/transactions_train.csv')
target_data = pd.read_csv('../data/age/train_target.csv')

In [5]:
def split_slice_subsample(sub_data, cnt_min, cnt_max, split_count):
    sub_datas = []
    for i in range(0, split_count):
        T_i = np.random.randint(cnt_min, cnt_max)
        s = np.random.randint(0, len(sub_data)-T_i-1)
        S_i = sub_data[s:s+T_i-1]
        sub_datas.append(S_i)
            
    return sub_datas

In [6]:
def create_set(name, data, target):
    len_ = len(np.unique(target.client_id))
    dict_data = {}
    with jsonlines.open(name, "w") as writer:
        S = 0
        for index, client_id in enumerate(np.unique(target.client_id)):
            sys.stdout.write("\r %d out of %d" % (index, len_))

            sub_data = data[data['client_id']==client_id]
            sub_data_target = target[target['client_id']==client_id]

            sub_datas = split_slice_subsample(sub_data, 50, 200, 30)

            for loc_data in sub_datas:
                loc_dict = {"transactions": list(loc_data.small_group),
                            "amounts": list(loc_data.amount_rur),
                            "label": int(sub_data_target.bins),
                            "client_id": int(client_id)}
                S = S+ len(loc_data.small_group)
                writer.write(loc_dict) 
                
    print('mean length:', S/(len(sub_datas)* len((np.unique(target.client_id)))))
    return

In [7]:
def split_data_lm(data, target_data):
    target_data_train, target_data_valid = train_test_split(target_data, test_size=0.2, random_state=10, shuffle=True)
    print('Create train set...')
    create_set('../../age/lm/train.jsonl', data, target_data_train)
    print('Create valid set...')
    create_set('../../age/lm/valid.jsonl', data, target_data_valid)
    return

In [8]:
split_data_lm(data, target_data)

Create train set...
 23999 out of 24000mean length: 85.98383472222223
Create valid set...
 5999 out of 6000mean length: 85.97567222222222


### Data for classifiers

In [9]:
!mkdir substitute_clf
!mkdir target_clf

mkdir: cannot create directory ‘substitute_clf’: File exists
mkdir: cannot create directory ‘target_clf’: File exists


In [10]:
target_data_test_sub, target_data_targetclf = train_test_split(target_data, test_size=0.65, random_state=10, shuffle=True)
target_data_subclf, target_data_test = train_test_split(target_data_test_sub, test_size=2./7, random_state=10, shuffle=True)

In [11]:
!mkdir substitute_clf
!mkdir target_clf
target_data_test_sub, target_data_targetclf = train_test_split(target_data, test_size=0.65, random_state=10, shuffle=True)
target_data_subclf, target_data_test = train_test_split(target_data_test_sub, test_size=2./7, random_state=10, shuffle=True)
def split_data(data, target_data, dir_):
    target_data_train, target_data_valid = train_test_split(target_data, test_size=0.2, random_state=10, shuffle=True)
    print('Create train set...')
    create_set(dir_+'/'+'train.jsonl', data, target_data_train)
    print('Create valid set...')
    create_set(str(dir_)+'/'+'valid.jsonl', data, target_data_valid)
    return

mkdir: cannot create directory ‘substitute_clf’: File exists
mkdir: cannot create directory ‘target_clf’: File exists


In [12]:
#create test set for both target and substitute classifiers
create_set('age_test.jsonl', data, target_data_test)

 2999 out of 3000mean length: 86.15154444444444


In [13]:
#create valid and train data for substitute classifier
split_data(data, target_data_subclf, '../../age/substitute_clf')

Create train set...
 5999 out of 6000mean length: 86.02505555555555
Create valid set...
 1499 out of 1500mean length: 86.15308888888889


In [14]:
#create valid and train data for target classifier
split_data(data, target_data_targetclf, '../../age/target_clf')

Create train set...
 15599 out of 15600mean length: 86.03875427350427
Create valid set...
 3899 out of 3900mean length: 85.93957264957265
